In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import sys
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
from datetime import datetime, timedelta

In [2]:
path="C:\\Users\\ninja\\Desktop\\6_proje_makalesi\\1_fall_detection\\proje dosyaları ve kodları\\veri_etiketleme_etl_pipeline\\veriler"
detections_path=path + "\\alp_sag_detection_dataDogru___20240821_175540_detections.csv"
imu_path=path +"\\alp_sag_imu_data_20240821_175456_imu_data.csv" # imu 


In [3]:
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
from datetime import timedelta
import sys

class ExtractFallData:
    def __init__(self):
        pass

    @staticmethod
    def ekf_filter(z, x_est, P, F, Q, H, R):
        x_pred = np.dot(F, x_est)  # State prediction
        P_pred = np.dot(F, np.dot(P, F.T)) + Q  # Covariance prediction

        # Measurement update
        y = z - np.dot(H, x_pred)  # Measurement residual
        S = np.dot(H, np.dot(P_pred, H.T)) + R  # Residual covariance
        K = np.dot(P_pred, np.dot(H.T, np.linalg.inv(S)))  # Kalman gain

        x_est_new = x_pred + np.dot(K, y)  # State update
        P_new = P_pred - np.dot(K, np.dot(H, P_pred))  # Covariance update

        return x_est_new, P_new

    @staticmethod
    def process_and_interpolate(filtered_df, desired_freq=50/1000):
        filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp'], errors='coerce')
        filtered_df.dropna(subset=['timestamp'], inplace=True)
        filtered_df['timestamp_unix'] = (filtered_df['timestamp'].astype(np.int64) // 10**6)
        filtered_df.sort_values(by='timestamp', inplace=True)
    
    
        filtered_df = filtered_df.drop_duplicates(subset=['timestamp_unix'])

        timestamps_unix = filtered_df['timestamp_unix'].values
        values = filtered_df[['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered']].values
        timestamps_unix, indices = np.unique(timestamps_unix, return_index=True)
        values = values[indices]   
        new_timestamps_unix = np.arange(timestamps_unix.min(), timestamps_unix.max(), 1 / desired_freq)
    
        try:
            cs = CubicSpline(timestamps_unix, values, axis=0, bc_type='natural')
            new_values = cs(new_timestamps_unix)
        except ValueError as e:
            print(f"Error creating cubic spline: {e}")
            print(f"Timestamps: {timestamps_unix}")
            print(f"Values: {values}")
            return None   

        interpolated_df = pd.DataFrame(new_values, columns=['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered'])
        interpolated_df['timestamp_unix'] = new_timestamps_unix

        return interpolated_df
    @staticmethod
    def apply_threshold(interpolated_df):
        columns_to_zero = ['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered']
        for column in columns_to_zero:
            if column in interpolated_df.columns:
                interpolated_df[column] = interpolated_df[column].where((interpolated_df[column] <= 20) & (interpolated_df[column] >= -20), 0)
        return interpolated_df

    @staticmethod
    def extract_imu_data(imu_df, x_est, P, F, Q, H, R):
        filtered_values = []
        for i in range(len(imu_df)):
            z = imu_df.iloc[i][['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values
            x_est, P = ExtractFallData.ekf_filter(z, x_est, P, F, Q, H, R)
            filtered_values.append(x_est)
        return pd.DataFrame(filtered_values, columns=['ax_filtered', 'ay_filtered', 'az_filtered', 'gx_filtered', 'gy_filtered', 'gz_filtered'])

    def two_esp_extractor(self, detections_path, imu_path):
        detections_df = pd.read_csv(detections_path)
        interpolated_imu = pd.read_csv(imu_path)
        
        imu_df_1 = interpolated_imu[interpolated_imu['topic'] == 'esp32_1/data'].sort_values(by='timestamp')
        imu_df_2 = interpolated_imu[interpolated_imu['topic'] == 'esp32_2/data'].sort_values(by='timestamp')

        F = np.eye(6)
        H = np.eye(6)
        Q = np.eye(6) * 0.01
        R = np.eye(6) * 0.1
        P = np.eye(6)
        x_est = np.zeros(6)

        filtered_df_1 = self.extract_imu_data(imu_df_1, x_est, P, F, Q, H, R)
        filtered_df_1['timestamp'] = imu_df_1['timestamp'].reset_index(drop=True)

        filtered_df_2 = self.extract_imu_data(imu_df_2, x_est, P, F, Q, H, R)
        filtered_df_2['timestamp'] = imu_df_2['timestamp'].reset_index(drop=True)

        interpolated_df_1 = self.process_and_interpolate(filtered_df_1)
        interpolated_df_2 = self.process_and_interpolate(filtered_df_2)

        interpolated_imu_1 = self.apply_threshold(interpolated_df_1)
        interpolated_imu_2 = self.apply_threshold(interpolated_df_2)

        
        interpolated_imu_1['timestamp'] = pd.to_datetime(interpolated_imu_1['timestamp_unix'], unit='ms')
        interpolated_imu_2['timestamp'] = pd.to_datetime(interpolated_imu_2['timestamp_unix'], unit='ms')
        first_timestamp = detections_df['Timestamp'].iloc[0]
        last_timestamp = detections_df['Timestamp'].iloc[-1]

        
        interpolated_imu_1 = interpolated_imu_1[(interpolated_imu_1['timestamp'] > first_timestamp) & (interpolated_imu_1['timestamp'] < last_timestamp)]
        interpolated_imu_2 = interpolated_imu_2[(interpolated_imu_2['timestamp'] > first_timestamp) & (interpolated_imu_2['timestamp'] < last_timestamp)]

        detections_df['Timestamp'] = pd.to_datetime(detections_df['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

        
        interpolated_imu_1['Class Name'] = 'not_falling'
        self.label_falling_periods(interpolated_imu_1, detections_df)
        self.label_falling_periods(interpolated_imu_2, detections_df)

        return interpolated_imu_1, interpolated_imu_2, detections_df

    @staticmethod
    def label_falling_periods(interpolated_imu, detections_df):
        count = 0
        falling_start_time = None

        for index, row in detections_df.iterrows():
            if row['Class Name'] == 'falling':
                count += 1
                if count == 5:
                    falling_start_time = detections_df.iloc[index - 4]['Timestamp']  
                    start_time = falling_start_time - timedelta(seconds=0.8)  
                    interpolated_imu.loc[(interpolated_imu['timestamp'] >= start_time) & (interpolated_imu['timestamp'] <= falling_start_time), 'Class Name'] = 'falling'
                    interpolated_imu.loc[(interpolated_imu['timestamp'] > falling_start_time), 'Class Name'] = 'not_falling'
                    count = 0 
            else:
                count = 0

        falling_start_time = None
        for index, row in interpolated_imu.iterrows():
            if row['Class Name'] == 'falling':
                if falling_start_time is None:
                    falling_start_time = row['timestamp']
                elif row['timestamp'] - falling_start_time > timedelta(seconds=1.6):
                    interpolated_imu.at[index, 'Class Name'] = 'not_falling'
            else:
                falling_start_time = None


In [4]:
etl = ExtractFallData()

In [5]:
interpolated_imu_1, interpolated_imu_2, detections_df = etl.two_esp_extractor(detections_path, imu_path)

C:\Users\ninja\AppData\Local\Temp\ipykernel_39716\4023519061.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interpolated_imu_1['Class Name'] = 'not_falling'
C:\Users\ninja\AppData\Local\Temp\ipykernel_39716\4023519061.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interpolated_imu.loc[(interpolated_imu['timestamp'] >= start_time) & (interpolated_imu['timestamp'] <= falling_start_time), 'Class Name'] = 'falling'


In [6]:
interpolated_imu_2

,ax_filtered,ay_filtered,az_filtered,gx_filtered,gy_filtered,gz_filtered,timestamp_unix,timestamp,Class Name
206,7.584612,10.132272,0.463267,1.621311,0.701167,-0.930295,1.724263e+12,2024-08-21 17:55:44.888,falling
207,7.446307,10.359714,0.512082,1.468659,0.778727,-0.958439,1.724263e+12,2024-08-21 17:55:44.908,falling
208,7.172568,10.530567,0.537091,1.313954,0.889188,-0.972525,1.724263e+12,2024-08-21 17:55:44.928,falling
209,6.815641,10.685932,0.524652,1.149797,1.038288,-0.971165,1.724263e+12,2024-08-21 17:55:44.948,falling
210,6.402705,10.773990,0.397876,1.020254,1.180785,-0.953356,1.724263e+12,2024-08-21 17:55:44.968,falling
...,...,...,...,...,...,...,...,...,...
8844,-5.026528,-5.062301,-10.102765,-1.571690,-1.652236,2.247382,1.724263e+12,2024-08-21 17:58:37.648,not_falling
8845,-5.152721,-4.135279,-10.261009,-1.458750,-1.378814,2.098301,1.724263e+12,2024-08-21 17:58:37.668,not_falling
8846,-5.882477,-2.932780,-11.091636,-1.340353,-1.018545,1.914958,1.724263e+12,2024-08-21 17:58:37.688,not_falling
8847,-6.300267,-2.165874,-11.474019,-1.237328,-0.747149,1.728567,1.724263e+12,2024-08-21 17:58:37.708,not_falling


In [7]:
interpolated_imu_1

,ax_filtered,ay_filtered,az_filtered,gx_filtered,gy_filtered,gz_filtered,timestamp_unix,timestamp,Class Name
207,-5.419303,9.167630,-3.488300,0.378283,1.364569,-0.568936,1.724263e+12,2024-08-21 17:55:44.887000064,falling
208,-4.788715,9.865896,-1.894273,0.629358,0.962854,-0.512865,1.724263e+12,2024-08-21 17:55:44.907000064,falling
209,-3.786127,10.067715,-1.538152,0.734812,0.910025,-0.511990,1.724263e+12,2024-08-21 17:55:44.927000064,falling
210,-3.539241,9.976071,-2.155057,0.800648,0.892626,-0.501407,1.724263e+12,2024-08-21 17:55:44.947000064,falling
211,-4.015271,9.757013,-2.805410,0.885365,0.730984,-0.443543,1.724263e+12,2024-08-21 17:55:44.967000064,falling
...,...,...,...,...,...,...,...,...,...
10277,19.019658,3.071444,-0.915498,-1.604479,-10.690152,2.674269,1.724263e+12,2024-08-21 17:59:06.287000064,not_falling
10278,8.294459,5.444997,3.059237,-0.825387,-6.321310,-0.537672,1.724263e+12,2024-08-21 17:59:06.307000064,not_falling
10279,-0.003505,7.521016,2.347053,-0.449788,-2.443959,-2.099323,1.724263e+12,2024-08-21 17:59:06.327000064,not_falling
10280,-2.745932,7.642096,-2.204985,-0.385891,-0.468596,-0.473108,1.724263e+12,2024-08-21 17:59:06.347000064,not_falling


In [8]:
detections_df

,Timestamp,Class Name,Confidence,Track ID
0,2024-08-21 17:55:44.876,falling,0.345433,NaN
1,2024-08-21 17:55:44.926,falling,0.479454,NaN
2,2024-08-21 17:55:44.976,falling,0.290418,NaN
3,2024-08-21 17:55:45.026,falling,0.515806,NaN
4,2024-08-21 17:55:45.126,falling,0.116733,NaN
...,...,...,...,...
3782,2024-08-21 17:59:06.176,not_falling,0.590795,3.0
3783,2024-08-21 17:59:06.226,not_falling,0.334215,3.0
3784,2024-08-21 17:59:06.276,not_falling,0.358066,3.0
3785,2024-08-21 17:59:06.326,not_falling,0.160431,3.0
